In [0]:
import pandas as pd
import warnings
import time
warnings.simplefilter(action='ignore', category=FutureWarning)
dbutils.widgets.text(label="Source Directory", name="source_path", defaultValue='')


In [0]:
schema_name = spark.catalog.currentDatabase()
catalog_name = spark.conf.get("spark.sql.catalogImplementation")

# print(f"Schema Name: {schema_name}")
# print(f"Catalog Name: {catalog_name}")

In [0]:
%sql
use catalog dev_edh;
use schema default;
--select count(*) from dev_edh.default.delta_source;
--truncate TABLE dev_poc.default.delta_source;


In [0]:
data = []  # Initialize an empty list to store data
cols = ['FileName', 'SheetName', 'ModificationTime', 'LoadTime', 'Data']  # Define column names for the data
directory = 'dbfs:/user/arundhuti/delta/'  # Define the directory path in DBFS

# Read the existing data to get the list of already processed files
existing_df = spark.read.format("parquet").load("dbfs:/user/arundhuti/delta/aru_delta_source")
existing_files = existing_df.select("FileName").distinct().rdd.flatMap(lambda x: x).collect()

# Iterate over each item in the specified directory
for item in dbutils.fs.ls(directory):
    # Check if the item is an Excel file with .xlsx extension and not already processed
    if item.path[-5:] == '.xlsx' and item.path[5:] not in existing_files:
        # Copy the file from DBFS to the local file system
        local_path = f"/tmp/{item.name}"
        dbutils.fs.cp(item.path, f"file:{local_path}")
        
        # Read the Excel file to get sheet names
        sheets = pd.ExcelFile(local_path)
        # Print the sheet names in the Excel file
        if len(sheets.sheet_names) >= 1:
            for sheet in sheets.sheet_names:
                pdf = pd.read_excel(local_path, sheet_name=sheet)
                modfied_time = int(item.modificationTime/1000)
                value = pdf.to_json(orient='split')
                load_time = time.strftime("%Y%m%d:%H:%M:%S", time.localtime())      
                data.append([item.path[5:], sheet, time.strftime('%Y%m%d:%H:%M:%S', time.localtime(modfied_time)), load_time, value])
                df = spark.createDataFrame(data, cols)      
                df.write.format("parquet").mode("append").save("dbfs:/user/arundhuti/delta/aru_delta_source")

In [0]:
%sql
SELECT * FROM PARQUET.`dbfs:/user/arundhuti/delta/aru_delta_source`

In [0]:
%sql
SELECT * FROM PARQUET.`dbfs:/user/arundhuti/delta/aru_delta_source` 
WHERE loadtime = (SELECT MAX(loadtime) FROM PARQUET.`dbfs:/user/arundhuti/delta/aru_delta_source`)

In [0]:
dbutils.fs.mv('dbfs:/FileStore/read_Jan.xlsx','dbfs:/user/arundhuti/delta/')